<a href="https://colab.research.google.com/github/leolellisr/deep_learning_projects/blob/main/10_LanguageModel_with_Self_Attention3/10_LanguageModel_with_Self_Attention3_unesp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
nome = "Leonardo de Lellis Rossi RA261900"
print(f'Meu nome é {nome}')

last = '22/06/04_23h56'
print(f'Last update: {last}')


Meu nome é Leonardo de Lellis Rossi RA261900
Last update: 22/06/04_23h56


#  Exercício - Aula 10: Refeito Modelo de Linguagem com auto-atenção

Este exercício é similar ao da Aula 8, mas iremos agora treinar uma rede neural com **duas camadas** de auto-atenção **causais** para prever a próxima palavra de um texto, data as palavras anteriores como entrada. 

Iremos também trabalhar com sequencias de tamanho variável.

Na camada de auto-atenção, não se esqueça de implementar:
- Embeddings de posição
- Projeções lineares (WQ, WK, WV, WO)
- Conexões residuais
- Camada de feed forward (2-layer MLP)


O dataset usado neste exercício (BrWaC) possui um tamanho razoável e você vai precisar rodar seus experimentos com GPU.

Alguns conselhos úteis:
- **ATENÇÃO:** o dataset é bem grande. Não dê comando de imprimí-lo.
- Durante a depuração, faça seu dataset ficar bem pequeno, para que a depuração seja mais rápida e não precise de GPU. Somente ligue a GPU quando o seu laço de treinamento já está funcionando
- Não deixe para fazer esse exercício na véspera. Ele é trabalhoso.

In [2]:
# iremos utilizar a biblioteca dos transformers para ter acesso ao tokenizador do BERT.
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 32.3 MB/s 
     |████████████████████████████████| 596 kB 57.3 MB/s 
     |████████████████████████████████| 6.6 MB 62.4 MB/s 
     |████████████████████████████████| 86 kB 6.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Importação dos pacotes

In [3]:
import collections
import itertools
import functools
import math
import random

import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader
from tqdm import tqdm_notebook


In [4]:
debug = False
checkinpoint = True
save_in_drive = True

params = {
    'max_examples': 150_000_000,
    'eval_every_steps': 10000,
    'lr': 3e-4,
    'batch_size': 128,
    'embedding_dim': 128,
    'hidden_size': 2*128,
    'optimizer': 'Adam',
    'retrain': True,
    'path_saved_model': 'gdrive/MyDrive/Colab Notebooks/best_model_',
    'path_saved_datasets': 'gdrive/MyDrive/Colab Notebooks/ds_',
    'download_ds': False,
    'aula': 'Aula10',
    'max_seq_length': 9,
    'train_examples': 90_000,
    'valid_examples': 40_000,
    'test_examples': 25_000,
    'n_heads':4,
    'last_step': 470000
}
params['path_saved_model'] = params['path_saved_model']+params['aula']+'_BS'+str(params['batch_size'])+'_HS'+str(params['hidden_size'])+'_EmbDim'+str(params['embedding_dim'])+'_MaxEx'+str(params['max_examples'])+'.pt'

In [5]:
# Check which GPU we are using
!nvidia-smi

Sun Jun  5 11:19:39 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [6]:
def set_seeds():
  random.seed(123)
  np.random.seed(123)
  torch.manual_seed(123)
  torch.cuda.manual_seed(123)
set_seeds()

# Neptune config

In [7]:
 !pip install -U neptune-client
 import neptune.new as neptune

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 317 kB 31.4 MB/s 
     |████████████████████████████████| 829 kB 55.6 MB/s 
     |████████████████████████████████| 54 kB 3.4 MB/s 
     |████████████████████████████████| 181 kB 66.3 MB/s 
     |████████████████████████████████| 132 kB 66.2 MB/s 
     |████████████████████████████████| 79 kB 10.1 MB/s 
     |████████████████████████████████| 8.8 MB 49.9 MB/s 
     |████████████████████████████████| 138 kB 59.2 MB/s 
     |████████████████████████████████| 63 kB 1.1 MB/s 
     |████████████████████████████████| 127 kB 55.0 MB/s 
     |████████████████████████████████| 67 kB 5.4 MB/s 
     |████████████████████████████████| 130 kB 71.8 MB/s 
     |████████████████████████████████| 64 kB 3.0 MB/s 
  Created wheel for neptune-client: filename=neptune_client-0.16.3-py2.py3-none-any.whl size=570148 sha256=1a9d984213fda7123f7393c3b5a1a580cafa225347f2ffa05

In [8]:
run = neptune.init(name= params['aula'], tags=[params['aula'], 'Auto-atenção', 'Self-Attention', 'checkinpoint', 'CrossEntropy', 'Adam', 'perplexity', 'BrWaC'],
    project="leolellisr/dl-ia025",
    api_token="eyJhcGlfYWRkcmVzcyI6Imh0dHBzOi8vYXBwLm5lcHR1bmUuYWkiLCJhcGlfdXJsIjoiaHR0cHM6Ly9hcHAubmVwdHVuZS5haSIsImFwaV9rZXkiOiI1NjY1YmJkZi1hYmM5LTQ3M2QtOGU1ZC1iZTFlNWY4NjE1NDQifQ==",
)

https://app.neptune.ai/leolellisr/dl-ia025/e/DLIA-128
Remember to stop your run once you’ve finished logging your metadata (https://docs.neptune.ai/api-reference/run#.stop). It will be stopped automatically only when the notebook kernel/interactive console is terminated.


In [9]:
run['parameters'] = params

In [10]:
import time
import itertools

## Implementação do MyDataset

In [11]:
from typing import List


def tokenize(text: str, tokenizer):
    return tokenizer(text, return_tensors=None, add_special_tokens=False).input_ids


class MyDataset():
    def __init__(self, texts: List[str], tokenizer, max_seq_length: int):
        self.max_seq_length = max_seq_length
        self.X = []
        for text in tqdm_notebook(texts):
          token_ids = tokenize(f'[CLS] {text}', tokenizer) # + [tokenizer.vocab['[SEP]']]  
          token_ids += [tokenizer.vocab['[PAD]']] * max(0, 1 + max_seq_length - len(token_ids))
          for i in range(0, len(token_ids) - 1, max_seq_length):
            if i + max_seq_length < len(token_ids):
              self.X.append(token_ids[i: i + max_seq_length + 1])
            else:
              self.X.append(token_ids[-max_seq_length - 1:])
        self.X = torch.LongTensor(self.X)

    def __len__(self):
        # Escreva seu código aqui
        return len(self.X)

    def __getitem__(self, idx):
        # Escreva seu código aqui
        x_y_idx = self.X[idx]
        return x_y_idx[:-1], x_y_idx[1:]       

In [12]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("neuralmind/bert-base-portuguese-cased")


Downloading:   0%|          | 0.00/205k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/43.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/647 [00:00<?, ?B/s]

## Testando se a implementação do MyDataset está correta

In [13]:
debug = False
params['batch_size'] = 10
batch_size = params['batch_size'] 
dummy_texts = ['Eu gosto de correr', 'Ela gosta muito de comer pizza']

dummy_dataset = MyDataset(texts=dummy_texts, tokenizer=tokenizer, max_seq_length=9)
dummy_loader = DataLoader(dummy_dataset, batch_size=6, shuffle=False)
print(f'len(dummy_dataset): {len(dummy_dataset)}')
assert len(dummy_dataset) == 2
print('Passou no assert de tamanho do dataset.')

first_batch_input, first_batch_target = next(iter(dummy_loader))

correct_first_batch_input = torch.LongTensor(
    [[  101,  3396, 10303,   125, 13239,     0,     0,     0,     0],
     [  101,  1660,  5971,   785,   125,  1847, 13779, 15616,     0]])

correct_first_batch_target = torch.LongTensor(
    [[ 3396, 10303,   125, 13239,     0,     0,     0,     0,     0],
     [ 1660,  5971,   785,   125,  1847, 13779, 15616,     0,     0]])


if debug: print(first_batch_input)

assert torch.equal(first_batch_input, correct_first_batch_input)

if debug:  print(first_batch_target)
assert torch.equal(first_batch_target, correct_first_batch_target)

print('Passou no assert de dataset.')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/2 [00:00<?, ?it/s]

len(dummy_dataset): 2
Passou no assert de tamanho do dataset.
Passou no assert de dataset.


In [14]:
from google.colab import drive

if save_in_drive: drive.mount('/content/gdrive')

Mounted at /content/gdrive


# Carregamento do dataset 

Iremos usar uma pequena amostra do dataset [BrWaC](https://www.inf.ufrgs.br/pln/wiki/index.php?title=BrWaC) para treinar e avaliar nosso modelo de linguagem.

In [15]:
!wget -nc https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt

--2022-06-05 11:20:58--  https://storage.googleapis.com/unicamp-dl/ia025a_2022s1/aula9/sample-1gb.txt
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.24.128, 142.251.10.128, 142.251.12.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.24.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1230909256 (1.1G) [text/plain]
Saving to: ‘sample-1gb.txt’

sample-1gb.txt      100%[===================>]   1.15G  72.1MB/s    in 17s     

2022-06-05 11:21:16 (70.5 MB/s) - ‘sample-1gb.txt’ saved [1230909256/1230909256]



In [16]:
# Load datasets
max_seq_length = params['max_seq_length']

train_examples = params['train_examples']
valid_examples = params['valid_examples']
test_examples = params['test_examples']

 

In [17]:
max_lines = train_examples + valid_examples + test_examples
print(f'Truncating to {max_lines} lines.')

if params['download_ds']:
  texts = open('sample-1gb.txt').readlines()
  print(f'Read {len(texts)} lines.')
  
  #smart batching
  texts = sorted(texts, key=lambda x: len(x[0]))

  texts = texts[:max_lines]  
  
  training_texts = texts[:-(valid_examples + test_examples)]
  valid_texts = texts[-(valid_examples + test_examples):-test_examples]
  test_texts = texts[-test_examples:]

  training_dataset = MyDataset(texts=training_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
  valid_dataset = MyDataset(texts=valid_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
  test_dataset = MyDataset(texts=test_texts, tokenizer=tokenizer, max_seq_length=max_seq_length)
else:
  path = params['path_saved_datasets']+'train.pt'
  training_dataset = torch.load(path)
  path = params['path_saved_datasets']+'val.pt'
  valid_dataset = torch.load(path)
  path = params['path_saved_datasets']+'test.pt'
  test_dataset = torch.load(path)
params['batch_size'] = 64
batch_size = params['batch_size'] 


Truncating to 155000 lines.


In [18]:
print(f'training examples: {len(training_dataset)}')
print(f'valid examples: {len(valid_dataset)}')
print(f'test examples: {len(test_dataset)}')

training examples: 11198040
valid examples: 5011771
test examples: 3041508


In [19]:

if save_in_drive and params['download_ds']:
  torch.save(training_dataset, params['path_saved_datasets']+"train.pt")
  torch.save(valid_dataset, params['path_saved_datasets']+"val.pt")
  torch.save(test_dataset, params['path_saved_datasets']+"test.pt")

In [20]:
import torch
import torch.nn.functional as F

In [21]:
# É recomendado reiniciar as seeds antes de inicializar o modelo, pois assim
# garantimos que os pesos vao ser sempre os mesmos.
set_seeds()

class SelfAttentionLayer(torch.nn.Module):

    def __init__(self, padding_idx, n_heads, dim, max_length):
        super().__init__()
        # n_heads: H
        # dim: D
        # max_lenght: L
        # vocab_lenght: V
        self.H = n_heads
        self.D = dim
        self.L = max_length
        self.D_H = self.D // self.H # D / H
        self.pad = padding_idx

        self.W_q = torch.nn.Linear(self.D, self.D, bias=False) # (D, D)
        self.W_k = torch.nn.Linear(self.D, self.D, bias=False)
        self.W_v = torch.nn.Linear(self.D, self.D, bias=False)
        self.W_o = torch.nn.Linear(self.D, self.D, bias=False)

        self.layer_norm1  = torch.nn.LayerNorm(self.D, eps=1e-6)

        self.feed_forward = torch.nn.Sequential(
            torch.nn.Linear(self.D, self.D*10),  
            torch.nn.ReLU(),
            torch.nn.Linear(self.D*10, self.D)
        )
        self.layer_norm2  = torch.nn.LayerNorm(self.D, eps=1e-6)


    def forward(self, x, att_mask):
        # multi-head self-attention
        
        fQ = self.W_q(x).reshape(len(x), self.L, self.H, self.D_H) # (B, L, H, D/H)
        fK = self.W_k(x).reshape(len(x), self.L, self.H, self.D_H)
        fV = self.W_v(x).reshape(len(x), self.L, self.H, self.D_H)

        # (B, L, H, D/H) -> (B, H, L, D/H)
        fQ_transposed = fQ.transpose(1, 2)                    # (B, H, L, D/H)
        fK_transposed = fK.transpose(1, 2)
        fV_transposed = fV.transpose(1, 2)

        scores = torch.matmul(fQ_transposed, fK_transposed.transpose(-2, -1)) / math.sqrt(self.D_H) # (B, H, L, L)

        scores = scores.masked_fill(att_mask.unsqueeze(1) == self.pad, -float("inf"))

        if debug: print(f"scores (B, H, L, L): {scores.shape}")  

        probs = F.softmax(scores, dim=-1) # shape = B, L, L
        if debug: print(f"probs (B, H, L, L): {probs.shape}")  

        E = torch.matmul(probs, fV_transposed)
        if debug: print(f"E (B, H, L, D/H): {E.shape}")  

        out = E.transpose(1, 2).contiguous()                   # (B, L, H, D/H)
        out = out.reshape(len(x), self.L, self.D) # (B, L, D) 
        out = self.W_o(out)
        if debug: print(f"out (B, L, D): {out.shape}")  # (B, L, D)    

        out = self.layer_norm1(x+out)               # (B, L, D) 
        out = self.feed_forward(out)
        out = self.layer_norm2(x+out)           # (B, L, D) 
        #att_mean = att_norm2 * padMask.unsqueeze(-1)      
        #mean_embeddings = att_mean.sum(dim=1) / padMask.count_nonzero(-1).unsqueeze(1)  
        return out

In [22]:
set_seeds()
class LanguageModel(torch.nn.Module):

    def __init__(self, vocab_size: int, max_seq_length: int, dim: int, n_layers: int, pad_token_id: int, hidden: int, n_heads):
        """
        Implements the Self-attention, decoder-only."

        Args:
            vocab_size (int): Size of the input vocabulary.
            max_seq_length (int): Size of the sequence to consider as context for prediction.
            dim (int): Dimension of the embedding layer for each word in the context.
            n_layers (int): number of self-attention layers.
            pad_token_id (int): id of the pad token that will be ignored in the attention.
          
        """
        # Escreva seu código aqui.
        super().__init__()
        self.H = n_heads 
        self.D = dim
        self.L = max_seq_length
        self.D_H = self.D // self.H # D / H
        self.pad_token_id = pad_token_id
        self.V = vocab_size
        self.hidden_size = hidden
        # word embedding 
        self.embeddings_c = nn.Embedding(self.V, self.D,padding_idx=pad_token_id)
        self.embeddings_p = torch.nn.Linear(self.D, self.L, bias=False)

#        self.att_layer = 
        self.att_layer1 = SelfAttentionLayer(padding_idx=pad_token_id, n_heads=self.H, dim=self.D, max_length=self.L)
        self.att_layer2 = SelfAttentionLayer(padding_idx=pad_token_id, n_heads=self.H, dim=self.D, max_length=self.L)

        self.feed_forward = torch.nn.Sequential(
            torch.nn.Linear(self.D, self.hidden_size),    # (D, hidden_size)
            torch.nn.ReLU(),
            torch.nn.Dropout(p=0.2),
            torch.nn.Linear(self.hidden_size, self.V)     # (hidden_size, V)
        )
        

        
    def forward(self, inputs):
        """
        Args:
            inputs is a LongTensor of shape (batch_size, max_seq_length)
            B: batch_size
            L: max_seq_length
            D: embedding_dim
            V: vocab_size
            input shape: (B, L)
            pos shape: (B, L)
        Returns:
            logits of shape (batch_size, max_seq_length, vocab_size)
        """
        B = inputs.shape[0]

        att_mask = torch.tril(torch.ones(B, self.L, self.L)).to(device)
        att_mask = att_mask.masked_fill(inputs.unsqueeze(1) == self.pad_token_id, 0)
        att_mask = att_mask.masked_fill(inputs.unsqueeze(2) == self.pad_token_id, 0)
        # input shape: (B, L)
        x_emb = self.embeddings_c(inputs) + self.embeddings_p.weight # (B, L, D)
        
        if debug: print(f'shape x_emb: {x_emb.shape}')                # (B, L, D)

        x_emb = self.att_layer1(x_emb, att_mask)     
        x_emb = self.att_layer2(x_emb, att_mask)
        #logits =  self.att_layer(x_emb, att_mask)                     # (B, L, D)
        if debug: print(f'shape x_emb: {x_emb.shape}')              # (B, L, D)
        logits = self.feed_forward(x_emb)                            # (B, L, V)
        if debug: print(f'shape logits (B, L, V): {logits.shape}')    # (B, L, V)
        

        return logits

In [23]:
if torch.cuda.is_available(): 
   dev = "cuda:0"
else: 
   dev = "cpu"
device = torch.device(dev)
print('Using {}'.format(device))

Using cuda:0


## Teste o modelo com um exemplo

In [24]:
debug = True
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=max_seq_length,
    dim=64,
    n_layers=2,
    hidden = 128,
    n_heads = 4,
    pad_token_id=tokenizer.pad_token_id
).to(device)


sample_input, _ = next(iter(DataLoader(training_dataset)))
sample_input = sample_input.to(device)
sample_output = model(sample_input)
print(f'sample_input.shape: {sample_input.shape}')
print(f'sample_output.shape: {sample_output.shape}')

shape x_emb: torch.Size([1, 9, 64])
scores (B, H, L, L): torch.Size([1, 4, 9, 9])
probs (B, H, L, L): torch.Size([1, 4, 9, 9])
E (B, H, L, D/H): torch.Size([1, 4, 9, 16])
out (B, L, D): torch.Size([1, 9, 64])
scores (B, H, L, L): torch.Size([1, 4, 9, 9])
probs (B, H, L, L): torch.Size([1, 4, 9, 9])
E (B, H, L, D/H): torch.Size([1, 4, 9, 16])
out (B, L, D): torch.Size([1, 9, 64])
shape x_emb: torch.Size([1, 9, 64])
shape logits (B, L, V): torch.Size([1, 9, 29794])
sample_input.shape: torch.Size([1, 9])
sample_output.shape: torch.Size([1, 9, 29794])


In [25]:
sample_input

tensor([[  101, 20100,  2308,  3074,  1089,   481,   117,   146,  1189]],
       device='cuda:0')

In [26]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'Number of model parameters: {num_params}')

Number of model parameters: 5957666


## Assert da Perplexidade


In [27]:
random.seed(123)
np.random.seed(123)
torch.manual_seed(123)


def perplexity(logits, target, ignore_token_id: int):
    """
    Computes the perplexity.

    Args:
        logits: a FloatTensor of shape (batch_size, seq_length, vocab_size)
        target: a LongTensor of shape (batch_size, seq_length)

    Returns:
        A float corresponding to the perplexity
    """
    logits = logits.reshape(-1, logits.shape[-1])
    target = target.reshape(-1)
    if debug: 
      print(f'logits: {logits.shape}')
      print(f'train_target_ids: {target.shape}')
    loss = nn.functional.cross_entropy(logits, target, reduction='mean', ignore_index=ignore_token_id)
    return torch.exp(loss)


n_examples = 1000

train_input_ids, train_target_ids = next(iter(DataLoader(training_dataset, batch_size=n_examples)))
train_input_ids = train_input_ids.to(device)
train_target_ids = train_target_ids.to(device)

logits = model.forward(train_input_ids)
if debug:
  print(f'logits: {logits.shape}')
  print(f'train_target_ids: {train_target_ids.shape}')

my_perplexity = perplexity(logits=logits, target=train_target_ids, ignore_token_id=tokenizer.pad_token_id)
if debug:
  print(f'correct initial perplexity: {tokenizer.vocab_size}')
  print(f'my perplexity:              {int(my_perplexity)}')


assert math.isclose(my_perplexity, tokenizer.vocab_size, abs_tol=7000)
print('Passou o no assert da perplexidade')
run['perplexity'].log(my_perplexity) # Envia perplexity para o Neptune.


shape x_emb: torch.Size([1000, 9, 64])
scores (B, H, L, L): torch.Size([1000, 4, 9, 9])
probs (B, H, L, L): torch.Size([1000, 4, 9, 9])
E (B, H, L, D/H): torch.Size([1000, 4, 9, 16])
out (B, L, D): torch.Size([1000, 9, 64])
scores (B, H, L, L): torch.Size([1000, 4, 9, 9])
probs (B, H, L, L): torch.Size([1000, 4, 9, 9])
E (B, H, L, D/H): torch.Size([1000, 4, 9, 16])
out (B, L, D): torch.Size([1000, 9, 64])
shape x_emb: torch.Size([1000, 9, 64])
shape logits (B, L, V): torch.Size([1000, 9, 29794])
logits: torch.Size([1000, 9, 29794])
train_target_ids: torch.Size([1000, 9])
logits: torch.Size([9000, 29794])
train_target_ids: torch.Size([9000])
correct initial perplexity: 29794
my perplexity:              30478
Passou o no assert da perplexidade


## Laço de Treinamento e Validação

In [28]:
batch_size = params['batch_size']

debug = False
model = LanguageModel(
    vocab_size=tokenizer.vocab_size,
    max_seq_length=params['max_seq_length'],
    dim=params['embedding_dim'],
    n_layers=2,
    hidden = params['hidden_size'],
    n_heads = params['n_heads'],
    pad_token_id=tokenizer.pad_token_id
).to(device)

train_loader = DataLoader(training_dataset, batch_size=params['batch_size'], shuffle=True, drop_last=True)
validation_loader = DataLoader(valid_dataset, batch_size=params['batch_size'])

optimizer = torch.optim.Adam(model.parameters(), lr=params['lr'])

best_valid_ppl = 10e9

if params['retrain']:
  model.load_state_dict(torch.load(params['path_saved_model']).state_dict())

def train_step(input_ids, target_ids):
    model.train()
    model.zero_grad()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    loss.backward()
    optimizer.step()

    return loss.item()


def validation_step(input_ids, target_ids):
    model.eval()
    logits = model(input_ids)
    logits = logits.reshape(-1, logits.shape[-1])
    target_ids = target_ids.reshape(-1)
    loss = nn.functional.cross_entropy(logits, target_ids, ignore_index=model.pad_token_id)
    return loss.item()

In [ ]:



train_losses = []
n_examples = 0
step = 0
while n_examples < params['max_examples']:
    for train_input_ids, train_target_ids in train_loader:
      if step < params['last_step']:
        step += 1
        n_examples += len(train_input_ids)  # Increment of batch size
      else:
        loss = train_step(train_input_ids.to(device), train_target_ids.to(device)) 
        train_losses.append(loss)
        
        if step %  params['eval_every_steps'] == 0:
            train_ppl = np.exp(np.average(train_losses))
            run['train/ppl'].log(train_ppl) # Envia train ppl para o Neptune.

            with torch.no_grad():
                valid_ppl = np.exp(np.average([
                    validation_step(val_input_ids.to(device), val_target_ids.to(device))
                    for val_input_ids, val_target_ids in validation_loader]))
            run['valid/ppl'].log(valid_ppl) # Envia valid ppl para o Neptune.
            if checkinpoint and valid_ppl < best_valid_ppl:
              torch.save(model.state_dict(), 'best_model.pt')
              if save_in_drive: torch.save(model, params['path_saved_model'])
              print(f"Best model found in step {step}. valid ppl: {valid_ppl:.2f}, best_valid_ppl: {best_valid_ppl:.2f} ")
              best_valid_ppl = valid_ppl
            ex_least = n_examples/params['max_examples']*100
            print(f'{step} steps; {n_examples} examples so far; {ex_least:.2f} % ; train ppl: {train_ppl:.2f}, valid ppl: {valid_ppl:.2f}, best_valid_ppl: {best_valid_ppl:.2f}')
            train_losses = []

        n_examples += len(train_input_ids)  # Increment of batch size
        step += 1
        if n_examples >= params['max_examples']:
            break

Best model found in step 470000. valid ppl: 155.89, best_valid_ppl: 10000000000.00 
470000 steps; 30080000 examples so far; 20.05 % ; train ppl: 159.47, valid ppl: 155.89, best_valid_ppl: 155.89
480000 steps; 30720000 examples so far; 20.48 % ; train ppl: 163.42, valid ppl: 157.81, best_valid_ppl: 155.89
490000 steps; 31360000 examples so far; 20.91 % ; train ppl: 163.54, valid ppl: 156.74, best_valid_ppl: 155.89
500000 steps; 32000000 examples so far; 21.33 % ; train ppl: 163.28, valid ppl: 155.90, best_valid_ppl: 155.89
Best model found in step 510000. valid ppl: 155.34, best_valid_ppl: 155.89 
510000 steps; 32640000 examples so far; 21.76 % ; train ppl: 162.80, valid ppl: 155.34, best_valid_ppl: 155.34
Best model found in step 520000. valid ppl: 155.26, best_valid_ppl: 155.34 
520000 steps; 33280000 examples so far; 22.19 % ; train ppl: 162.51, valid ppl: 155.26, best_valid_ppl: 155.26
Best model found in step 530000. valid ppl: 154.74, best_valid_ppl: 155.26 
530000 steps; 33920000

## Avaliação final no dataset de teste


Bonus: o modelo com menor perplexidade no dataset de testes ganhará 0.5 ponto na nota final.

In [29]:
if params['retrain']:
  model.load_state_dict(torch.load(params['path_saved_model']).state_dict())

In [30]:
#best_model = 'best_model.pt'


test_loader = DataLoader(test_dataset, batch_size=params['batch_size'])

with torch.no_grad():
    test_ppl = np.exp(np.average([
        validation_step(test_input_ids.to(device), test_target_ids.to(device))
        for test_input_ids, test_target_ids in test_loader
    ]))

print(f'test perplexity: {test_ppl}')

test perplexity: 149.7980049617075


In [31]:
run['test/perplexity'].log(test_ppl)


In [32]:
run.stop()

Shutting down background jobs, please wait a moment...
Done!
Waiting for the remaining 4 operations to synchronize with Neptune. Do not kill this process.
All 4 operations synced, thanks for waiting!
Explore the metadata in the Neptune app:
https://app.neptune.ai/leolellisr/dl-ia025/e/DLIA-128


## Teste seu modelo com uma sentença

Escolha uma sentença gerada pelo modelo que ache interessante.

In [33]:
prompt = 'Eu gosto de comer pizza pois me faz'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    print(f'pred: {predicted_id}')
    print(f'input_ids: {input_ids}')
    prompt = tokenizer.decode(input_ids)
    print(prompt)

pred: 10500
input_ids: [3396, 10303, 125, 1847, 13779, 15616, 1502, 311, 659, 10500]
Eu gosto de comer pizza pois me faz sentir
pred: 146
input_ids: [3396, 10303, 125, 1847, 13779, 15616, 1502, 311, 659, 10500, 146]
Eu gosto de comer pizza pois me faz sentir o
pred: 179
input_ids: [3396, 10303, 125, 1847, 13779, 15616, 1502, 311, 659, 10500, 146, 179]
Eu gosto de comer pizza pois me faz sentir o que
pred: 2779
input_ids: [3396, 10303, 125, 1847, 13779, 15616, 1502, 311, 659, 10500, 146, 179, 2779]
Eu gosto de comer pizza pois me faz sentir o que eu
pred: 346
input_ids: [3396, 10303, 125, 1847, 13779, 15616, 1502, 311, 659, 10500, 146, 179, 2779, 346]
Eu gosto de comer pizza pois me faz sentir o que eu não
pred: 311
input_ids: [3396, 10303, 125, 1847, 13779, 15616, 1502, 311, 659, 10500, 146, 179, 2779, 346, 311]
Eu gosto de comer pizza pois me faz sentir o que eu não me
pred: 5069
input_ids: [3396, 10303, 125, 1847, 13779, 15616, 1502, 311, 659, 10500, 146, 179, 2779, 346, 311, 5069]
E

In [34]:
prompt = 'Ouviram do Ipiranga em suas margens plácidas um grito'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Ouviram do Ipiranga em suas margens plácidas um grito de
Ouviram do Ipiranga em suas margens plácidas um grito de um
Ouviram do Ipiranga em suas margens plácidas um grito de um lado
Ouviram do Ipiranga em suas margens plácidas um grito de um lado,
Ouviram do Ipiranga em suas margens plácidas um grito de um lado, o
Ouviram do Ipiranga em suas margens plácidas um grito de um lado, o que
Ouviram do Ipiranga em suas margens plácidas um grito de um lado, o que é
Ouviram do Ipiranga em suas margens plácidas um grito de um lado, o que é o
Ouviram do Ipiranga em suas margens plácidas um grito de um lado, o que é o que
Ouviram do Ipiranga em suas margens plácidas um grito de um lado, o que é o que é
Ouviram do Ipiranga em suas margens plácidas um grito de um lado, o que é o que é o
Ouviram do Ipiranga em suas margens plácidas um grito de um lado, o que é o que é o que
Ouviram do Ipiranga em suas margens plácidas um grito de um lado, o que é o que é o que é
Ouviram do Ipiranga em suas margens pl

In [35]:
prompt ='A galinha atravessou a rua para chegar'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

A galinha atravessou a rua para chegar a
A galinha atravessou a rua para chegar a um
A galinha atravessou a rua para chegar a um dia
A galinha atravessou a rua para chegar a um dia.
A galinha atravessou a rua para chegar a um dia. O
A galinha atravessou a rua para chegar a um dia. O que
A galinha atravessou a rua para chegar a um dia. O que é
A galinha atravessou a rua para chegar a um dia. O que é o
A galinha atravessou a rua para chegar a um dia. O que é o que
A galinha atravessou a rua para chegar a um dia. O que é o que é
A galinha atravessou a rua para chegar a um dia. O que é o que é o
A galinha atravessou a rua para chegar a um dia. O que é o que é o que
A galinha atravessou a rua para chegar a um dia. O que é o que é o que é
A galinha atravessou a rua para chegar a um dia. O que é o que é o que é o
A galinha atravessou a rua para chegar a um dia. O que é o que é o que é o que
A galinha atravessou a rua para chegar a um dia. O que é o que é o que é o que é
A galinha atravessou a

In [36]:
prompt ='Ouça com cuidado, o segredo para a felicidade é'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Ouça com cuidado, o segredo para a felicidade é a
Ouça com cuidado, o segredo para a felicidade é a de
Ouça com cuidado, o segredo para a felicidade é a de que
Ouça com cuidado, o segredo para a felicidade é a de que o
Ouça com cuidado, o segredo para a felicidade é a de que o que
Ouça com cuidado, o segredo para a felicidade é a de que o que é
Ouça com cuidado, o segredo para a felicidade é a de que o que é o
Ouça com cuidado, o segredo para a felicidade é a de que o que é o que
Ouça com cuidado, o segredo para a felicidade é a de que o que é o que é
Ouça com cuidado, o segredo para a felicidade é a de que o que é o que é o
Ouça com cuidado, o segredo para a felicidade é a de que o que é o que é o que
Ouça com cuidado, o segredo para a felicidade é a de que o que é o que é o que é
Ouça com cuidado, o segredo para a felicidade é a de que o que é o que é o que é o
Ouça com cuidado, o segredo para a felicidade é a de que o que é o que é o que é o que
Ouça com cuidado, o segredo para a fe

In [37]:
prompt ='Temos que pegar! Isso eu sei. Pegá-los eu tentarei! Vai ser grande a'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Temos que pegar! Isso eu sei. Pegá - los eu tentarei! Vai ser grande a minha
Temos que pegar! Isso eu sei. Pegá - los eu tentarei! Vai ser grande a minha vida
Temos que pegar! Isso eu sei. Pegá - los eu tentarei! Vai ser grande a minha vida.
Temos que pegar! Isso eu sei. Pegá - los eu tentarei! Vai ser grande a minha vida..
Temos que pegar! Isso eu sei. Pegá - los eu tentarei! Vai ser grande a minha vida...
Temos que pegar! Isso eu sei. Pegá - los eu tentarei! Vai ser grande a minha vida....
Temos que pegar! Isso eu sei. Pegá - los eu tentarei! Vai ser grande a minha vida.....
Temos que pegar! Isso eu sei. Pegá - los eu tentarei! Vai ser grande a minha vida......
Temos que pegar! Isso eu sei. Pegá - los eu tentarei! Vai ser grande a minha vida.......
Temos que pegar! Isso eu sei. Pegá - los eu tentarei! Vai ser grande a minha vida........
Temos que pegar! Isso eu sei. Pegá - los eu tentarei! Vai ser grande a minha vida.........
Temos que pegar! Isso eu sei. Pegá - los eu tentarei! Vai 

In [38]:
prompt ='Desejo a todas as inimigas vida longa'
max_output_tokens = 20
model.eval()

for _ in range(max_output_tokens):
    input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    prompt = tokenizer.decode(input_ids)
    print(prompt)

Desejo a todas as inimigas vida longa.
Desejo a todas as inimigas vida longa. O
Desejo a todas as inimigas vida longa. O que
Desejo a todas as inimigas vida longa. O que é
Desejo a todas as inimigas vida longa. O que é o
Desejo a todas as inimigas vida longa. O que é o que
Desejo a todas as inimigas vida longa. O que é o que é
Desejo a todas as inimigas vida longa. O que é o que é o
Desejo a todas as inimigas vida longa. O que é o que é o que
Desejo a todas as inimigas vida longa. O que é o que é o que é
Desejo a todas as inimigas vida longa. O que é o que é o que é o
Desejo a todas as inimigas vida longa. O que é o que é o que é o que
Desejo a todas as inimigas vida longa. O que é o que é o que é o que é
Desejo a todas as inimigas vida longa. O que é o que é o que é o que é o
Desejo a todas as inimigas vida longa. O que é o que é o que é o que é o que
Desejo a todas as inimigas vida longa. O que é o que é o que é o que é o que é
Desejo a todas as inimigas vida longa. O que é o que é o

In [39]:
test = next(iter(test_loader))
input_ids = test[0][1]
max_output_tokens = 20
model.eval()
input_ids = input_ids.tolist()
prompt = tokenizer.decode(input_ids)
print(prompt)
for _ in range(max_output_tokens):
    #input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    #print(input_ids)
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    #print(input_ids)
    prompt = tokenizer.decode(input_ids)
    print(prompt)

##cio, chega ao Brasil numa tradução completa e
##cio, chega ao Brasil numa tradução completa e o
##cio, chega ao Brasil numa tradução completa e o que
##cio, chega ao Brasil numa tradução completa e o que é
##cio, chega ao Brasil numa tradução completa e o que é o
##cio, chega ao Brasil numa tradução completa e o que é o que
##cio, chega ao Brasil numa tradução completa e o que é o que é
##cio, chega ao Brasil numa tradução completa e o que é o que é o
##cio, chega ao Brasil numa tradução completa e o que é o que é o que
##cio, chega ao Brasil numa tradução completa e o que é o que é o que é
##cio, chega ao Brasil numa tradução completa e o que é o que é o que é o
##cio, chega ao Brasil numa tradução completa e o que é o que é o que é o que
##cio, chega ao Brasil numa tradução completa e o que é o que é o que é o que é
##cio, chega ao Brasil numa tradução completa e o que é o que é o que é o que é o
##cio, chega ao Brasil numa tradução completa e o que é o que é o que é o que é o que


In [40]:
test = next(iter(train_loader))
input_ids = test[0][1]
max_output_tokens = 20
model.eval()
input_ids = input_ids.tolist()
prompt = tokenizer.decode(input_ids)
print(prompt)
for _ in range(max_output_tokens):
    #input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    #print(input_ids)
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    #print(input_ids)
    prompt = tokenizer.decode(input_ids)
    print(prompt)

##jamentos, automaticamente, prorrogados, até
##jamentos, automaticamente, prorrogados, até o
##jamentos, automaticamente, prorrogados, até o dia
##jamentos, automaticamente, prorrogados, até o dia 12
##jamentos, automaticamente, prorrogados, até o dia 12 de
##jamentos, automaticamente, prorrogados, até o dia 12 de setembro
##jamentos, automaticamente, prorrogados, até o dia 12 de setembro,
##jamentos, automaticamente, prorrogados, até o dia 12 de setembro, o
##jamentos, automaticamente, prorrogados, até o dia 12 de setembro, o que
##jamentos, automaticamente, prorrogados, até o dia 12 de setembro, o que é
##jamentos, automaticamente, prorrogados, até o dia 12 de setembro, o que é o
##jamentos, automaticamente, prorrogados, até o dia 12 de setembro, o que é o primeiro
##jamentos, automaticamente, prorrogados, até o dia 12 de setembro, o que é o primeiro,
##jamentos, automaticamente, prorrogados, até o dia 12 de setembro, o que é o primeiro, o
##jamentos, automaticamente, prorrogados, a

In [41]:
test = next(iter(validation_loader))
input_ids = test[0][1]
max_output_tokens = 20
model.eval()
input_ids = input_ids.tolist()
prompt = tokenizer.decode(input_ids)
print(prompt)
for _ in range(max_output_tokens):
    #input_ids = tokenize(text=prompt, tokenizer=tokenizer)
    input_ids_truncated = input_ids[-max_seq_length:]  # Usamos apenas os últimos <max_seq_length> tokens como entrada para o modelo.
    logits = model(torch.LongTensor([input_ids_truncated]).to(device))
    #print(input_ids)
    logits = logits[:, -1, :]  # Usamos apenas o ultimo token da sequencia
    # Ao usarmos o argmax, a saída do modelo em cada passo é o token de maior probabilidade.
    # Isso se chama decodificação gulosa (greedy decoding).
    predicted_id = torch.argmax(logits).item()
    
    input_ids += [predicted_id]  # Concatenamos a entrada com o token escolhido nesse passo.
    #print(input_ids)
    prompt = tokenizer.decode(input_ids)
    print(prompt)

palestrante. Trabalhou como redator e diretor
palestrante. Trabalhou como redator e diretor da
palestrante. Trabalhou como redator e diretor da Universidade
palestrante. Trabalhou como redator e diretor da Universidade de
palestrante. Trabalhou como redator e diretor da Universidade de São
palestrante. Trabalhou como redator e diretor da Universidade de São Paulo
palestrante. Trabalhou como redator e diretor da Universidade de São Paulo,
palestrante. Trabalhou como redator e diretor da Universidade de São Paulo, que
palestrante. Trabalhou como redator e diretor da Universidade de São Paulo, que foi
palestrante. Trabalhou como redator e diretor da Universidade de São Paulo, que foi o
palestrante. Trabalhou como redator e diretor da Universidade de São Paulo, que foi o primeiro
palestrante. Trabalhou como redator e diretor da Universidade de São Paulo, que foi o primeiro a
palestrante. Trabalhou como redator e diretor da Universidade de São Paulo, que foi o primeiro a ser
palestrante. Tr

## Bonus 1
Quem conseguir a menor perplexidade no dataset de testes ganha 0.5 ponto na média final.

## Bonus 2
Qual é a complexidade (em notação O-grande) da função de geração de texto acima?

Quem responder corretamente a pergunta acima e deixar a função com menor complexidade ganha 0.5 ponto na média final.